In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms

import sklearn
from sklearn.metrics import classification_report

from tqdm import tqdm
import os

# Note

FC Layer 로 image 처리는 안되나? => parameter 수가 너무 많아짐, 2 차원 특징을 학습하기 힘듬   
    
transforms: 전처리 적용   
1. ToTensor: torch 텐서로 취급   
2. Normalize: trainset 의 평균, 분산으로 픽셀들의 range 를 scaling   

# Data Preparation

In [9]:
path = os.getenv("RESOURCE_PATH")

train_loader = DataLoader(datasets.FashionMNIST(path, train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])),
batch_size=64, shuffle=True)

test_loader = DataLoader(datasets.FashionMNIST(path, train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])),
batch_size=64, shuffle=True)

In [13]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.Layer = nn.Sequential(
            nn.Conv2d(1,32,3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*5*5, 128),
            nn.ReLU(),
            nn.Linear(128,10),
        )
    
    def forward(self,x):
        return self.Layer(x)
    
model = CNN()

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in tqdm(range(5)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1,1,28,28).to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()

100%|██████████| 5/5 [00:40<00:00,  8.06s/it]


In [19]:
total, correct = 0, 0
y_pred, y_true = torch.zeros([64]).to(device), torch.zeros([64]).to(device)

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1,1,28,28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        if(predicted.shape[0]!=64):
            predicted = torch.cat([predicted, torch.zeros(64-predicted.shape[0]).to(device)])
            labels = torch.cat([labels, torch.zeros(64-labels.shape[0]).to(device)])

        y_pred = torch.cat([y_pred, predicted])
        y_true = torch.cat([y_true, labels])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Test Accuracy: {}%".format((correct / total) * 100))

Test Accuracy: 90.11743630573248%


In [20]:
print(classification_report(y_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy()))

              precision    recall  f1-score   support

         0.0       0.90      0.83      0.86      1112
         1.0       1.00      0.97      0.98      1000
         2.0       0.91      0.78      0.84      1000
         3.0       0.92      0.90      0.91      1000
         4.0       0.82      0.90      0.85      1000
         5.0       0.98      0.98      0.98      1000
         6.0       0.66      0.78      0.72      1000
         7.0       0.94      0.98      0.96      1000
         8.0       0.97      0.98      0.98      1000
         9.0       0.99      0.94      0.96      1000

    accuracy                           0.90     10112
   macro avg       0.91      0.90      0.90     10112
weighted avg       0.91      0.90      0.90     10112

